In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ozon-masters-ds-contest-2021/sample_submission.csv
/kaggle/input/ozon-masters-ds-contest-2021/train.csv
/kaggle/input/ozon-masters-ds-contest-2021/test.csv


1) признак что "X2">"X4"

2) mean encoding 1 игрока относительно X2

3) mean encoding 2 игрока относительно X4

4) разность X2 и X4 и log от этой величины

5) log(X21)

6) выбираем где 'logX21'>=5.85

7) перевод в numpy и масштабируем признаки


In [2]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import StackingClassifier
import seaborn as sns
import time
import datetime
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss


In [3]:
d= pd.read_csv('/kaggle/input/ozon-masters-ds-contest-2021/train.csv')


In [4]:
cor = d.corr()
#cor.style.background_gradient(cmap='coolwarm')

In [5]:
def comp(d):
    if d['X2'] > d['X4']:
        return 1
    else:
        return 0
d['comp'] = d.apply(lambda d :comp(d),axis=1)
d

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X14,X15,X16,X17,X18,X19,X20,X21,target,comp
0,0,2,908749,954,480270,935,71,79,53,89,...,100,87,3,61,48,43,79,329,0,1
1,1,1,438444,2162,486685,2154,32,39,48,49,...,83,91,6,57,94,15,89,581,1,1
2,2,1,596915,2066,711059,2081,43,21,58,94,...,52,37,50,69,60,89,10,548,0,0
3,3,1,625198,1508,66810,1474,7,1,34,73,...,27,55,97,83,1,88,87,555,1,1
4,4,2,228654,1202,542816,1196,53,35,16,80,...,95,50,55,10,49,12,68,651,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,87,92,70,16,80,52,89,615,1,1
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,10,94,27,64,82,79,34,517,1,0
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,5,86,49,79,74,87,48,279,0,1
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1,37,7,88,39,83,32,669,0,0


In [6]:
y = d['target'].values
x = d.drop('target',axis=1)
train_new = d
skf = StratifiedKFold(n_splits=2,shuffle=True, random_state=241)
cols = ['X1','X5','X6','X7','X8','X9','X10','X11','X12']
for tr_ind, val_ind in skf.split(x,y):
    x_tr, x_val = d.iloc[tr_ind], d.iloc[val_ind]
    for col in cols:
        means = x_val[col].map(x_tr.groupby(col)['X2'].mean())
        x_val[col+'_mean_target'] = means
    
train_new['X1_mean_X2'] = x_val['X1_mean_target']
train_new['X5_mean_X2'] = x_val['X5_mean_target']
train_new['X6_mean_X2'] = x_val['X6_mean_target']
train_new['X7_mean_X2'] = x_val['X7_mean_target']
train_new['X8_mean_X2'] = x_val['X8_mean_target']
train_new['X9_mean_X2'] = x_val['X9_mean_target']
train_new['X10_mean_X2'] = x_val['X10_mean_target']
train_new['X11_mean_X2'] = x_val['X11_mean_target']
train_new['X12_mean_X2'] = x_val['X12_mean_target']
prior = d['X2'].mean()
train_new.fillna(prior, inplace = True)
d = train_new


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
y = d['target'].values
x = d.drop('target',axis=1)
train_new = d
skf = StratifiedKFold(n_splits=2,shuffle=True, random_state=241)
cols = ['X3','X13','X14','X15','X16','X17','X18','X19','X20']
for tr_ind, val_ind in skf.split(x,y):
    x_tr, x_val = d.iloc[tr_ind], d.iloc[val_ind]
    for col in cols:
        means = x_val[col].map(x_tr.groupby(col)['X4'].mean())
        x_val[col+'_mean_target'] = means
    #print(x_val['X0_mean_target'])
train_new['X3_mean_X4'] = x_val['X3_mean_target']
train_new['X13_mean_X4'] = x_val['X13_mean_target']
train_new['X14_mean_X4'] = x_val['X14_mean_target']
train_new['X15_mean_X4'] = x_val['X15_mean_target']
train_new['X16_mean_X4'] = x_val['X16_mean_target']
train_new['X17_mean_X4'] = x_val['X17_mean_target']
train_new['X18_mean_X4'] = x_val['X18_mean_target']
train_new['X19_mean_X4'] = x_val['X19_mean_target']
train_new['X20_mean_X4'] = x_val['X20_mean_target']
prior = d['X4'].mean()
train_new.fillna(prior, inplace = True)
d = train_new


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
a = d['X2'] - d['X4']
d['dif'] = a
d

#d['dif'].value_counts()


,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X3_mean_X4,X13_mean_X4,X14_mean_X4,X15_mean_X4,X16_mean_X4,X17_mean_X4,X18_mean_X4,X19_mean_X4,X20_mean_X4,dif
0,0,2,908749,954,480270,935,71,79,53,89,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,19
1,1,1,438444,2162,486685,2154,32,39,48,49,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,8
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-15
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,34
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1349.600000,1317.348972,1401.587889,1283.205923,1548.517365,1388.961479,1276.528348,1721.451109,1532.802625,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,17
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,835.250000,1488.946398,1384.221648,1506.491841,1705.084637,1079.118417,1554.906226,1187.496316,1404.591915,-7
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,1124.400000,1640.487502,1406.548609,1139.477104,1275.137128,1188.001467,1351.403367,1525.229217,1488.075525,1
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-28


In [9]:
q = np.log(d['X21']+0.01)
d['logX21'] = q
d

,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X13_mean_X4,X14_mean_X4,X15_mean_X4,X16_mean_X4,X17_mean_X4,X18_mean_X4,X19_mean_X4,X20_mean_X4,dif,logX21
0,0,2,908749,954,480270,935,71,79,53,89,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,19,5.796088
1,1,1,438444,2162,486685,2154,32,39,48,49,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,8,6.364768
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-15,6.306294
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,34,6.318986
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1317.348972,1401.587889,1283.205923,1548.517365,1388.961479,1276.528348,1721.451109,1532.802625,6,6.478525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,17,6.421639
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,1488.946398,1384.221648,1506.491841,1705.084637,1079.118417,1554.906226,1187.496316,1404.591915,-7,6.248062
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,1640.487502,1406.548609,1139.477104,1275.137128,1188.001467,1351.403367,1525.229217,1488.075525,1,5.631248
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-28,6.505799


In [10]:
t = np.log(d['dif']+410)
d['logdif'] = t
d['logdif'].isna().sum()
#d.shape
#pr = d['logdif'].mean()
#d.fillna(pr,inplace = True)
d


,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X14_mean_X4,X15_mean_X4,X16_mean_X4,X17_mean_X4,X18_mean_X4,X19_mean_X4,X20_mean_X4,dif,logX21,logdif
0,0,2,908749,954,480270,935,71,79,53,89,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,19,5.796088,6.061457
1,1,1,438444,2162,486685,2154,32,39,48,49,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,8,6.364768,6.035481
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-15,6.306294,5.978886
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,34,6.318986,6.095825
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1401.587889,1283.205923,1548.517365,1388.961479,1276.528348,1721.451109,1532.802625,6,6.478525,6.030685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,17,6.421639,6.056784
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,1384.221648,1506.491841,1705.084637,1079.118417,1554.906226,1187.496316,1404.591915,-7,6.248062,5.998937
3633002,3633002,2,227914,1063,174332,1062,48,64,55,3,...,1406.548609,1139.477104,1275.137128,1188.001467,1351.403367,1525.229217,1488.075525,1,5.631248,6.018593
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-28,6.505799,5.945421


In [11]:
#sns.histplot(d['X21'])
a = d.loc[d['logX21']>=5.85] #& ((d['dif'] > -250) & (d['dif'] < 250))]
a
#aa = pd.concat([a,q])
#aa['X21>=350'].value_counts()
#aa


,id,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X14_mean_X4,X15_mean_X4,X16_mean_X4,X17_mean_X4,X18_mean_X4,X19_mean_X4,X20_mean_X4,dif,logX21,logdif
1,1,1,438444,2162,486685,2154,32,39,48,49,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,8,6.364768,6.035481
2,2,1,596915,2066,711059,2081,43,21,58,94,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-15,6.306294,5.978886
3,3,1,625198,1508,66810,1474,7,1,34,73,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,34,6.318986,6.095825
4,4,2,228654,1202,542816,1196,53,35,16,80,...,1401.587889,1283.205923,1548.517365,1388.961479,1276.528348,1721.451109,1532.802625,6,6.478525,6.030685
5,5,1,1102751,1172,833352,1178,5,64,78,45,...,1300.465033,1570.592294,1434.661407,1576.905464,1470.201276,1535.083937,1441.270596,-6,6.304467,6.001415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3632999,3632999,2,816875,944,496602,970,63,95,91,62,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-26,6.450486,5.950643
3633000,3633000,2,644871,1211,421763,1194,24,65,54,86,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,17,6.421639,6.056784
3633001,3633001,1,689210,699,569184,706,26,50,81,33,...,1384.221648,1506.491841,1705.084637,1079.118417,1554.906226,1187.496316,1404.591915,-7,6.248062,5.998937
3633003,3633003,2,1066558,1752,358523,1780,10,31,47,68,...,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,1450.096512,-28,6.505799,5.945421


In [12]:
#qr = a.drop(['id','X2','X4','X1','X3','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','dif'], axis = 1)
#qr.shape
#cor = qr.corr()
#cor.style.background_gradient(cmap='coolwarm')

In [13]:
yy = a['target']
y = yy.to_numpy()
xxx = a.drop(['id','target','X2','X4','X1','X3','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','dif'], axis = 1)
#xxx
xx = xxx.to_numpy()
#xx.shape
#xa = np.concatenate((xx,qw),axis=1)

scal = StandardScaler()
x = scal.fit_transform(xx)

#pca = PCA()
#x = pca.fit_transform(X)
x.shape

(2966076, 22)

In [14]:
cval = KFold(n_splits=2, shuffle=True, random_state=1)
su = 0
start_time = datetime.datetime.now()
for train, test in cval.split(x, y):
    start_time = datetime.datetime.now()
    x_train, x_test, y_train, y_test = x[train], x[test], y[train], y[test]
    #model = RandomForestClassifier()
    #model = LogisticRegression(penalty='l2', C=3)
    #model = LGBMClassifier(learning_rate=0.3,max_depth=4)
    #model = XGBClassifier(learning_rate=0.2,max_depth=4,min_child_weight=500)
    level0 = list()
    level0.append(('lr', LogisticRegression(penalty='l2', C=3)))
    level0.append(('lgbm', LGBMClassifier(learning_rate=0.3,max_depth=4)))
    level0.append(('xgb', XGBClassifier(learning_rate=0.2,max_depth=4,min_child_weight=500)))

    level1 = XGBClassifier(learning_rate=0.2,max_depth=4,min_child_weight=500)
    
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=2)
    
    print('1')
    model.fit(x_train, y_train)
    #print(model.feature_importances_)
    #pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
    #pyplot.show()
    print('1')
    pred = model.predict_proba(x_test)[:, 1]
    sc = log_loss(y_test, pred)
    su = su + sc
    print('Time elapsed for :', '2', datetime.datetime.now() - start_time)
    print('\n')

print('качество работы алгоритма ', su / 2.0)
print('\n')

1


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:10:34] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:12:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:07] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:13:45] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1
Time elapsed for : 2 0:04:18.875967


1


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:14:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:16:48] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:17:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:18:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1
Time elapsed for : 2 0:04:19.367923


качество работы алгоритма  0.6584664338127744




In [15]:
test_data= pd.read_csv('/kaggle/input/ozon-masters-ds-contest-2021/test.csv')
test_data1 = test_data.drop(['id'],axis =1)
test_data1.shape


(908252, 22)

In [16]:
test_new = test_data1

cols = ['X1','X5','X6','X7','X8','X9','X10','X11','X12']
for col in cols:
    means = test_new[col].map(test_new.groupby(col)['X2'].mean())
    test_new[col+'_mean_X2'] = means

prior = test_new['X2'].mean()
test_new.fillna(prior, inplace = True)
d = test_new
d.shape


(908252, 31)

In [17]:
test_new = test_data1

cols = ['X3','X13','X14','X15','X16','X17','X18','X19','X20']
for col in cols:
    means = test_new[col].map(test_new.groupby(col)['X4'].mean())
    test_new[col+'_mean_X4'] = means
    
prior = test_new['X4'].mean()
test_new.fillna(prior, inplace = True)
d = test_new
d.shape

(908252, 40)

In [18]:
#test_new = test_data1
#cols = ['X0','X1','X3','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20']
#for col in cols:
    #means = test_new[col].map(test_new.groupby(col)['X21'].mean())
    #test_new[col+'_mean_X21'] = means
    
#prior = test_new['X21'].mean()
#test_new.fillna(prior, inplace = True)
#d = test_new
#d.shape

In [19]:
a = d['X2'] - d['X4']
d['dif'] = a


In [20]:
q = np.log(d['X21']+0.01)
d['logX21'] = q


In [21]:
t = np.log(d['dif']+410)
d['logdif'] = t


In [22]:
def comp(d):
    if d['X2'] > d['X4']:
        return 1
    else:
        return 0
d['comp'] = d.apply(lambda d :comp(d),axis=1)
d.head()

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15_mean_X4,X16_mean_X4,X17_mean_X4,X18_mean_X4,X19_mean_X4,X20_mean_X4,dif,logX21,logdif,comp
0,1,1032217,1187,1011523,1172,57,100,1,80,25,...,1314.317919,1388.214259,1441.354269,1439.118542,1393.821436,1383.105143,15,6.304467,6.052089,1
1,2,1059033,2128,505263,2104,82,27,87,79,80,...,1458.762905,1488.719142,1371.382832,1390.825900,1404.411580,1576.271798,24,6.486176,6.073045,1
2,1,207787,1017,822639,1040,82,48,43,62,18,...,1325.763304,1104.134538,1298.175834,1381.417430,1463.577785,1311.357653,-23,6.079956,5.958425,0
3,1,1095582,1449,32841,1424,71,43,11,98,80,...,1509.349157,1417.956274,1529.663980,1312.725204,1436.118742,1482.031348,25,6.378443,6.075346,1
4,2,963764,1202,489871,1182,11,62,16,55,88,...,1577.119447,1480.870154,1441.354269,1493.430333,1274.519581,1525.344146,20,5.828975,6.063785,1


In [23]:
xxx = d.drop(['X2','X4','X1','X3','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','dif'], axis = 1)
xx = xxx.to_numpy()
#xx.shape


scal = StandardScaler()
xt = scal.fit_transform(xx)
#pca = PCA()
#x = pca.fit_transform(x)
xt.shape

(908252, 22)

In [24]:
level0 = list()
level0.append(('lr', LogisticRegression(penalty='l2', C=3)))
level0.append(('lgbm', LGBMClassifier(learning_rate=0.3,max_depth=4)))
level0.append(('xgb', XGBClassifier(learning_rate=0.2,max_depth=4,min_child_weight=500)))

level1 = XGBClassifier(learning_rate=0.2,max_depth=4,min_child_weight=500)
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=2)

model.fit(x, y)
print('1')
pred = model.predict_proba(xt)[:, 1]
print(pred[1000:1100])
print(pred.shape[0])

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:19:47] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:24:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:25:38] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:27:13] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1
[0.40725937 0.48647255 0.37433133 0.21010442 0.5259457  0.27877712
 0.4817437  0.3059996  0.32135582 0.23737316 0.31661284 0.2486851
 0.42931354 0.3149036  0.26648992 0.32416102 0.25819835 0.61506844
 0.33424255 0.24842045 0.4381976  0.300659

In [25]:
idx = np.arange(pred.shape[0])
list_of_tuples = list(zip(idx, pred))

df = pd.DataFrame(list_of_tuples,
                  columns = ['id', 'target'])
    

df.to_csv('submission.csv', index = False)